## 4.1

Für die Vorhersage von 'Average playtime forever' sind folgende Merkmale besonders relevant:

- Median playtime forever – stark mit dem Zielwert korreliert
- Price – kann mit Spielwert und Motivation in Zusammenhang stehen
- Positive – hohe Anzahl an positiven Bewertungen weist auf häufige und lange Spielnutzung hin

Diese Merkmale enthalten starke Signale, die das Modell zur Prognose nutzen kann.

## 4.2

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [2]:
games = pd.read_csv("/Users/ylli/Library/CloudStorage/OneDrive-AlteKantonsschuleAarau/BBBaden/M259/games.csv")
games.head()

AppID                   Name Release date Estimated owners  Required age  \
0    20200       Galactic Bowling    21-Oct-08        0 - 20000             0   
1   655370           Train Bandit    12-Oct-17        0 - 20000             0   
2  1732930           Jolt Project    17-Nov-21        0 - 20000             0   
3  1355720               Henosis™    23-Jul-20        0 - 20000             0   
4  1139950  Two Weeks in Painland     3-Feb-20        0 - 20000             0   

   Price                                Supported languages  Positive  \
0  19.99                                        ['English']         6   
1   0.99  ['English', 'French', 'Italian', 'German', 'Sp...        53   
2   4.99                 ['English', 'Portuguese - Brazil']         0   
3   5.99  ['English', 'French', 'Italian', 'German', 'Sp...         3   
4   0.00                     ['English', 'Spanish - Spain']        50   

   Negative  Achievements  Recommendations  Average playtime forever  \
0        11            30                0                         0   
1         5            12                0                         0   
2         0             0                0                         0   
3         0             0                0                         0   
4         8            17                0                         0   

              Publishers                                         Categories  \
0  Perpetual FX Creative  Single-player,Multi-player,Steam Achievements,...   
1           Wild Rooster  Single-player,Steam Achievements,Full controll...   
2          Campião Games                                      Single-player   
3      Odd Critter Games              Single-player,Full controller support   
4          Unusual Games                   Single-player,Steam Achievements   

                            Genres  
0              Casual,Indie,Sports  
1                     Action,Indie  
2  Action,Adventure,Indie,Strategy  
3           Adventure,Casual,Indie  
4                  Adventure,Indie

Messmetrik

Für das lineare Regressionsmodell wurden folgende Metriken berechnet:

- **MSE** (Mean Squared Error): durchschnittlich quadrierter Fehler

Diese Werte zeigen, wie weit die Vorhersagen im Durchschnitt von den tatsächlichen Spielzeiten abweichen.

In [3]:
# Daten vorbereiten
games = games.dropna(subset=["Average playtime forever", "Price", "Positive"])

X = games[["Price", "Positive"]]
y = games["Average playtime forever"]

# Trainings- und Testdaten splitten
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modell trainieren
model = LinearRegression()
model.fit(X_train, y_train)

# Vorhersagen berechnen
predictions = model.predict(X_test)


In [4]:
from sklearn.metrics import mean_squared_error
import numpy as np

mse = mean_squared_error(y_test, predictions)

print("📉 MSE (Mean Squared Error):", round(mse, 2))

📉 MSE (Mean Squared Error): 1058802.58


## 4.3

Um das Regressionsmodell klassifikatorisch auszuwerten, wurde ein Schwellenwert von 100 Minuten definiert: Spiele mit einer vorhergesagten oder tatsächlichen Spielzeit ab 100 Minuten gelten als Langspieler.

Basierend auf dieser Schwelle wurde eine Wahrheitsmatrix erstellt und die Kennzahlen Precision und Recall berechnet. Das Modell erreicht eine hohe Recall-Rate (0.75), was bedeutet, dass die meisten echten Langspieler korrekt erkannt wurden. Die Precision liegt jedoch nur bei 0.25 – viele vorhergesagte Langspieler waren in Wirklichkeit keine.

Das Modell erkennt Langspieler gut, neigt aber zur Überschätzung.

In [5]:
# Wir definieren: ab 100 Minuten zählt als "Langspieler" = Klasse 1
threshold = 100

# Tatsächliche Klassen
y_test_binary = (y_test >= threshold).astype(int)

# Vorhergesagte Klassen
predictions_binary = (predictions >= threshold).astype(int)

In [6]:
from sklearn.metrics import confusion_matrix

# Matrix berechnen
cm = confusion_matrix(y_test_binary, predictions_binary)
tn, fp, fn, tp = cm.ravel()

print("📊 Confusion Matrix:")
print(cm)
print(f"TN = {tn}, FP = {fp}, FN = {fn}, TP = {tp}")


📊 Confusion Matrix:
[[907 830]
 [ 91 272]]
TN = 907, FP = 830, FN = 91, TP = 272


In [7]:
from sklearn.metrics import precision_score, recall_score

precision = precision_score(y_test_binary, predictions_binary)
recall = recall_score(y_test_binary, predictions_binary)

print("🎯 Precision (Wie genau sind die Langspieler-Vorhersagen?):", round(precision, 2))
print("📈 Recall (Wie viele Langspieler wurden erkannt?):", round(recall, 2))

🎯 Precision (Wie genau sind die Langspieler-Vorhersagen?): 0.25
📈 Recall (Wie viele Langspieler wurden erkannt?): 0.75


## 4.4

Das Regressionsmodell liefert brauchbare Ergebnisse für die Vorhersage der durchschnittlichen Spielzeit. Die Metriken zeigen jedoch, dass die Vorhersagen bei Langspielern nicht besonders präzise sind.

Die nachträgliche Klassifizierung (Schwellenwert 100 Minuten) zeigt:
- **Recall** ist hoch (0.75), das Modell erkennt die meisten Langspieler.
- **Precision** ist niedrig (0.25), das Modell klassifiziert aber viele Spiele fälschlich als Langspieler.

Mögliche Gründe:
- Spielzeit ist stark variabel und nicht klar trennbar.
- Zusätzliche Merkmale wie `Genres`, `Achievements` oder `Recommendations` wurden nicht berücksichtigt.

Ein nicht-lineares Modell oder eine echte Klassifikation könnte bessere Ergebnisse liefern.